In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import argparse
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from csv import DictWriter
from threading import Thread
import os, csv, re, queue, time, winsound, traceback, selenium
import pycountry

chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--ignore-ssl-errors')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--incognito')
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument("--headless")
caps = webdriver.DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "normal"
service = Service(ChromeDriverManager().install())

LINK = 'https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&field1=AllField&text1=LEACH+AND+WSN+AND+ENERGY&pageSize=50&expand=all'

In [2]:
driver = webdriver.Chrome(service=service, options=chrome_options)
link ='https://www.mdpi.com/search?sort=pubdate&page_count=200&year_from=1996&year_to=2024&advanced=(%40(title%2Cabstract%2Ckeywords%2Cauthors%2Caffiliations%2Cdoi%2Cfull_text%2Creferences)LEACH%40(title%2Cabstract%2Ckeywords%2Cauthors%2Caffiliations%2Cdoi%2Cfull_text%2Creferences)energy%40(title%2Cabstract%2Ckeywords%2Cauthors%2Caffiliations%2Cdoi%2Cfull_text%2Creferences)WSN)&view=abstract'

driver.get(link)
allow_all_cookies_button = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
)

# Click the 'Allow all cookies' button
allow_all_cookies_button.click()
html = driver.page_source
driver.close()
soups = BeautifulSoup(html, 'html.parser')


In [3]:
row_data = {
        'title'                         : '0',
        'link'                          : '0',
        'number_of_citation'            : '0',
        'article_type'                  : '0',
        'publisher'                     : 'ACM',
        'keyword'                       : '0',
        'abstract'                      : '0',
        'publish_date'                  : '0',
        'publication_title'             : '0',
        'authors'                       : '0',
        'affiliations'                  : '0',
        'countries'                     : '0'
    }
    
for col in row_data.keys():
    print(f"{col}:\n\t\t\t\t{row_data[col]}")

title:
				0
link:
				0
number_of_citation:
				0
article_type:
				0
publisher:
				ACM
keyword:
				0
abstract:
				0
publish_date:
				0
publication_title:
				0
authors:
				0
affiliations:
				0
countries:
				0


In [4]:

for article in soups.find_all('div', class_='generic-item article-item')[:5]:
    driver = webdriver.Chrome(service=service, options=chrome_options)

    link_article   = article.find('div', class_='color-grey-dark').find('a').get_text()
    link_article   = 'https://doi.org/10.3390/s24206502'
    article_type   = article.find('span', class_='label articletype').get_text()
    article_title  = article.find('a', class_='title-link').get_text()
    driver.get(link_article)
    driver.implicitly_wait(5)

    # allow_all_cookies_button = None
    # # Click on 'Info' section with error handling
    # try:
    #     allow_all_cookies_button = driver.find_element(By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll")
        
    #     max_attempts = 3
    #     for attempt in range(max_attempts):
    #         try:
    #             allow_all_cookies_button = WebDriverWait(driver, 10).until(
    #                 EC.visibility_of_element_located((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
    #             )
    #             allow_all_cookies_button.click()
    #             break
    #         except selenium.common.exceptions.ElementClickInterceptedException:
    #             if attempt < max_attempts - 1:
    #                 print("Retrying click after a short delay...")
    #                 time.sleep(1)
    #             else:
    #                 print("Failed to click after multiple attempts.")
    #                 raise

    # except selenium.common.exceptions.TimeoutException:
    #     print("Element was not clickable within the given time.")
    #     continue
    # allow_all_cookies_button = WebDriverWait(driver, 10).until(
    #     EC.visibility_of_element_located((By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"))
    # )
    # allow_all_cookies_button.click()
    html = driver.page_source
    article_soup = BeautifulSoup(html, 'html.parser')
    publication_title = article_soup.find('div', class_='bib-identity').find('em').get_text()

    number_of_citation = article_soup.find('div', id='counts-wrapper').find('span', class_='count citations-number Var_ArticleMaxCitations')

    if number_of_citation:
        number_of_citation = number_of_citation.get_text()
    else:
        number_of_citation = 'none'
   
    keywords = article_soup.find_all('div', class_='html-gwd-group')
    if keywords != None:
        for keyword in keywords:
            keyword = keyword.find_all('a')
            keyword = ','.join([i.get_text() for i in  keyword])
        keywords = keyword
    else:
        keywords = 'none'

    abstract = article_soup.find('section', class_="html-abstract") 
    if abstract != None:
        abstract = abstract.get_text()
    else:
        abstract = 'none'

    publication_date = article_soup.find('div', class_='pubhistory').find_all('span')
    if len(publication_date) == 3:
        publication_date = publication_date[2].get_text()
        publication_date = publication_date.replace('Accepted: ', '')
    elif len(publication_date) == 4:
        publication_date = publication_date[2].get_text()
        publication_date = publication_date.replace('Published: ', '')
    
    publication_title = article_soup.find('div', class_='bib-identity').find('em').get_text()
    
    affiliation_info = {}
    for i in article_soup.find_all('div', class_="affiliation"):
        affiliation_info[i.find('sup').get_text()] = i.find('div', class_='affiliation-name').get_text()


    affiliation_info.pop('*')
    author_informations = article_soup.find('div', class_='art-authors hypothesis_container')
    author_informations = author_informations.find_all('span', class_="inlineblock")

    authors = ''
    countries = ''
    affiliations = ''
    count = 0

    for author_information in author_informations:
        author = author_information.find('span', class_='sciprofiles-link__name').get_text()
        index_affiliations = author_information.find('sup').get_text().split(',')
        for index_affiliation in index_affiliations:
            if index_affiliation != '*':
                index_affiliation = index_affiliation.replace(" ", "")
                affiliation = affiliation_info[index_affiliation]
                country = affiliation.split(',')
                countries += country[len(country)-1]

                affiliations +=  f'({affiliation})'

                affiliation = affiliation.split(', ')
                country = affiliation[len(affiliation)-1]
                countries       += ','
                affiliations    += ','
                    
        authors += author
        if count < len(author_informations)-1:
            authors         += ','
        count += 1
        occurrences = countries.split(',')
        found_countries = []

        for country in pycountry.countries:
            for occurrence in occurrences:
                if country.name in occurrence:
                    found_countries.append(country.name)

        countries = ','.join(found_countries)

        row_data = {
                'title'                         : article_title,
                'link'                          : link_article,
                'number_of_citation'            : number_of_citation,
                'article_type'                  : article_type,
                'publisher'                     : 'ACM',
                'keyword'                       : keywords,
                'abstract'                      : abstract,
                'publish_date'                  : publication_date,
                'publication_title'             : publication_title,
                'authors'                       : authors,
                'affiliations'                  : affiliations,
                'countries'                     : countries
            }
        
        for col in row_data.keys():
            print(f"{col}:\n\t\t\t\t{row_data[col]}")
    
    driver.close()
    print('\n\n\n')

    # except Exception as e:
    #     print("\n"*3)
    #     print("== ! Error ! =="*10)
    #     print("\n"*1)
    #     print(f"Failed to retrieve article: \n{article_title}\n{link_article}\n{e}")
    #     traceback.print_exc()
    #     print("\n"*3)
    #     print("== ! Error ! =="*10)
    #     print("\n"*1)


AttributeError: 'NoneType' object has no attribute 'get_text'

<div class="color-grey-dark">
<em>Sensors</em> <b>2024</b>, <em>24</em>(21), 6952; https://doi.org/10.3390/s24216952 (registering DOI) - 30 Oct 2024
    </div>

In [6]:
html = str(article.find('div', class_='color-grey-dark'))

# Parse the HTML
soup = BeautifulSoup(html, 'html.parser')

# Extract the text content of the div and find the DOI link
text = soup.get_text()
doi_link = re.search(r'https://doi\.org/\S+', text).group(0)
print(doi_link)

https://doi.org/10.3390/s24216952
